In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F

/Users/steve/steve/neural-language-models/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# read data

In [2]:
words = open('../data/names.txt', 'r').read().splitlines()

In [3]:
len(words)

32033

In [4]:
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [5]:
chars = sorted(list(set(''.join(words))))

# util

In [6]:
itos = {}
itos[0] = '.'
itos |= {i+1: s for i, s in enumerate(chars)}

In [7]:
stoi = {s: i for i, s in itos.items()}

# train a neural language model

## prepare data

In [9]:
X, Y = [], []
for w in words[:]:
    w2 = '.' + w + '.'
    for ch1, ch2 in zip(w2, w2[1:]):
        if ch1 != '.' and ch1 != 'a':
            continue
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        X.append(ix1)
        Y.append(ix2)

In [10]:
X = torch.tensor(X)
Y = torch.tensor(Y)